# Estimating Fault Tolerant Reseources for Periodic Systems

The resource estimation code provided in this module relies on pyscf to compute the required symmetry adapted molecular orbitals, one- and two-electron integrals, and, correlated wavefunction calculations to determine various thresholds. So to start the tutorial let's run a periodic restriced Hartree--Fock (RHF) simulation of diamond. 

We assume a knowledge of electronic structure theory for solids, and in particular an understanding that any results need to be converged to the infintie $k$-point and complete basis set limit. For the purposes of this tutorial we will focus on results far away from this limit and simulate simple systems in minimal basis sets use very small $k$-point meshes. The module also assumes the use of density fitted integrals and we use range-separated density fitting throughout. 

In [24]:
from ase.build import bulk
import numpy as np

from pyscf.pbc import gto, scf
from pyscf.pbc.tools import pyscf_ase


# Build a 2 atom unit cell for carbon in the diamond structure near it's
# equilibrium lattice constant.
ase_atom = bulk("C", "diamond", a=3.5)
cell = gto.Cell()
cell.atom = pyscf_ase.ase_atoms_to_pyscf(ase_atom)
cell.a = ase_atom.cell[:].copy()
# Using a minimal basis set for expediency.
cell.basis = "gth-szv"
cell.pseudo = "gth-hf-rev"
cell.verbose = 0
cell.build()

# We are using a very small k-point mesh for speed purposes too.
kmesh = [1, 1, 3]
kpts = cell.make_kpts(kmesh)
num_kpts = len(kpts)
mf = scf.KRHF(cell, kpts).rs_density_fit() # Use range separated density fitting.
mf.kernel()
print("SCF energy: ", mf.e_tot)

# converged SCF energy with: -10.39193609748544

SCF energy:  -10.39193609745196


Armed with our SCF solution we can now generate the one and two-electron integrals required to compute $\lambda$.

In [25]:
from kpoint_eri.factorizations.hamiltonian_utils import build_hamiltonian
# Get molecular orbital "Cholesky" integrals from RSGDF object, these are just
# 3-centre integrals (X|pq).
hcore_mo, LXpq = build_hamiltonian(mf)
print("(nkpts, nmo, nmo) = {}".format(hcore_mo.shape))
print("(nkpts, nkpts) = {}".format(LXpq.shape))
print("(naux, nmo, nmo) = {}".format(LXpq[0,0].shape))
num_mo = hcore_mo.shape[-1]
num_aux = LXpq[0,0].shape[0]

(nkpts, nmo, nmo) = (3, 8, 8)
(nkpts, nkpts) = (3, 3)
(naux, nmo, nmo) = (108, 8, 8)


# Hamiltonian Representation
The resource estimation module provides four representations for the Hamiltonian: sparse, single-factorization, double-factorization, and tensor hypercontraction (THC). Each of these approaches introduces a different parameter which controls the accuracy of the factorization. Recall that we have

## Sparse Hamiltonian

The sparse Hamiltonian takes the usual form for a second-quantized $k$-point dependent Hamiltonian

$$

H = \sum_{pq\mathrm{k}} h_{pq}(\mathrm{k}) a_{p\mathbf{k}}^{\dagger} a_{q\mathbf{k}} + \frac{1}{2} \sum_{\mathbf{k}_p\mathbf{k}_q\mathbf{k}_r\mathbf{k}_s}\sum_{pqrs} (p\mathbf{k}_pq\mathbf{k}_q|r\mathbf{k}_rs\mathbf{k}_s)  a_{p\mathbf{k}_p}^{\dagger} a_{r\mathbf{k}_r}^{\dagger} a_{s\mathbf{k}_s}a_{q\mathbf{k}_q} 

$$

Utilizing conservation of crystal momentum $\mathbf{k}_p + \mathbf{k}_r - \mathbf{k}_q -\mathbf{k}_s = \mathbf{G}$, where $\mathbf{G}$ is a reciprocal lattice vector we can write 

$$

H = \sum_{pq\mathrm{k}} h_{pq}(\mathrm{k}) a_{p\mathbf{k}}^{\dagger} a_{q\mathbf{k}} + \frac{1}{2} \sum_{\mathbf{Q}\mathbf{k}\mathbf{k}'}\sum_{pqrs} (p\mathbf{k}q\mathbf{k}-\mathbf{Q}|r\mathbf{k}'-\mathbf{Q}s\mathbf{k}')  a_{p\mathbf{k}}^{\dagger} a_{r\mathbf{k}'-\mathbf{Q}}^{\dagger} a_{s\mathbf{k}'}a_{q\mathbf{k}-\mathbf{Q}} 

$$

where $\mathbf{Q}$ is the momentum transfer vector which we chose to live in our set of $k$-points. Note the subtraction in the above expression is really modulo a $\mathbf{G}$ vector.

The Hamiltonian above has $N_k^3 N^4$ terms, where $N_k$ is the number of $k$-points and $N$ is the number of spin orbitals. The sparse representation attempts to approximate the Hamiltonian by zeroing elements of $H$ which are below some threshold. This will yield $\mathcal{O}(s N_k^3 N^4)$ terms where $s$ is some sparsity factor.

We provide helper functions that will sparsify the Hamiltonian in order to perform resource estimates.


In [26]:
from kpoint_eri.resource_estimates.sparse.integral_helper_sparse import SparseFactorizationHelper 
sparse_ham = SparseFactorizationHelper(LXpq, mf, threshold=1e-3)
# look at eri block
kpts = [0]*4
eri_approx = sparse_ham.get_eri(kpts)
eri_exact = sparse_ham.get_eri_exact(kpts)
assert not np.allclose(eri_approx, eri_exact)
print("Total number of elements N_k^3 N^4 = {:d}".format(num_kpts**3*num_mo**4))
print("number of symmetry unique non zero = {}".format(sparse_ham.get_total_unique_terms_above_thresh()))

Total number of elements N_k^3 N^4 = 110592
number of symmetry unique non zero = 17087


With the Hamiltonian in hand we can compute the 1-norm (called $\lambda$) of the Hamiltonian which is essential for computing resource estimates.

In [27]:
from kpoint_eri.resource_estimates.sparse.compute_lambda_sparse import compute_lambda 
sparse_lambda = compute_lambda(hcore_mo, sparse_ham)

Finally we can compute the total resource costs. In particular the following code will compute the number of Toffoli gates required for a single step of phase estimation (`toffolis_per_step`), the total Toffoli count (`total_toffolis`), and the number of logical qubits (`logical_qubits`).

The total Toffoli count is given by the toffoli per step cost multiplied by the factor 

$$

\left\lceil \frac{\pi \lambda}{2 \epsilon_{\mathrm{QPE}}} \right\rceil

$$

where $\epsilon_{\mathrm{QPE}}$ is our variable `dE_for_qpe`.

In [28]:
from kpoint_eri.resource_estimates.sparse.compute_sparse_resources import cost_sparse
num_spin_orbs = 2*num_mo 
resources = cost_sparse(num_spin_orbs, sparse_lambda.lambda_total, sparse_lambda.num_sym_unique , kmesh, dE_for_qpe=0.0016)
print(resources)

ResourceEstimates(toffolis_per_step=2764, total_toffolis=2633287676, logical_qubits=1143)


We see there are the following steps required:

1. Run an SCF calculation.
2. Generate the one- and two-electron matrix elements.
3. Compute the lambda value of the Hamiltonian. 
4. Compute the resource estimates.

These steps are required for all four factorizations and involve a lot of boilerplate code. As such we provide utility functions which perform these necessary steps, and optionally scan over the value of the corresponding threshold parameter.

Let's see how this works for the sparse factorization first.

In [29]:
from kpoint_eri.resource_estimates.sparse.generate_costing_table_sparse import generate_costing_table
thresholds = np.logspace(-1, -5, 6)
sparse_costing_table = generate_costing_table(mf, thresholds=thresholds)

TypeError: PBCResources.add_resources() got an unexpected keyword argument 'approx_energy'

In [ ]:
df = sparse_costing_table.to_dataframe()
print(df.to_string())

Let's look at the convergence of the MP2 error with the sparsity threshold.

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 130 
plt.plot(df.cutoff, np.abs(df.approx_energy-df.exact_energy), marker="o")
plt.xscale("log")
plt.yscale("log")
plt.xlabel("sparse threshold")
plt.ylabel("MP2 Energy Error (Ha)")

## Single Factorization 

The single factorization Hamiltonian takes the following form:

$$

(p\mathbf{k}q\mathbf{k}-\mathbf{Q}|r\mathbf{k}'-\mathbf{Q}s\mathbf{k}') = \sum_n^{M} L_{p\mathbf{k}q\mathbf{k}-\mathbf{Q}}^n L_{s\mathbf{k}'r\mathbf{k}'-\mathbf{Q}}^{n*} 


$$
where $M$ is the dimension of auxiliary index $n$. 

In [ ]:
from kpoint_eri.resource_estimates.sf.generate_costing_table_sf import generate_costing_table
cutoffs = np.linspace(10, naux, 8)
sparse_costing_table = generate_costing_table(mf, cutoffs=thresholds)
